In [1]:
#!/usr/bin/env python3

import os
import shutil
import subprocess

from pathlib import Path
from argparse import ArgumentParser

# 0. Set Environment Variables

In [ ]:
PASSWORD=123 

OPT_LIB="/opt"
USER_LIB="/usr/lib"
BASH_DIR="/home/fyj/.bashrc"
WORK_DIR="/home/fyj/Desktop/data/1_WCETSpace"
# DEBIAN_FRONTEND=noninteractive

# (0) build
BUILD_DIR=os.path.join(WORK_DIR, "build")

# (1) wcet tool
TAM_HOME=os.path.join(WORK_DIR, "TAM")
LLVMTA_HOME=os.path.join(WORK_DIR, "LLVMTA")

# (2) lp & gruobi 
LS_SDIR=os.path.join(f"{USER_LIB}", f"lp_solve", f"liblpsolve55.so")
LS_TDIR=os.path.join(f"{USER_LIB}",              f"liblpsolve55.so")

GUROBI_VERS=['12', '0', '3']
GUROBI_FILE=f"gurobi{''.join(GUROBI_VERS)}"
GUROBI_PACKAGE=f"gurobi{'.'.join(GUROBI_VERS)}_linux64.tar.gz"

GUROBI_HOME=os.path.join(OPT_LIB, GUROBI_FILE, f"linux64")
LD_LIBRARY_PATH=os.path.join(GUROBI_HOME, f"lib")
CPLUS_INCLUDE_PATH=os.path.join(GUROBI_HOME, f"include")

GB_SDIR=os.path.join(f"{LD_LIBRARY_PATH}",  f"libgurobi{''.join(GUROBI_VERS)}.so")
GB_TDIR=os.path.join(f"{USER_LIB}",         f"libgurobi.so")

# (3) llvm
LLVM_VERS=["14", "0", "6"]
LLVM_HOME=os.path.join(WORK_DIR, "LLVM")
# LLVM_HOME=os.path.join(f"{WORK_DIR}", f"llvm-project")

LLVM_DIR=os.path.join(LLVM_HOME, "llvm")
CLANG_DIR=os.path.join(LLVM_HOME, "clang")

In [ ]:
PAGES=[
    "build-essential",
    "ccache",
    "clangd",
    "clang-tidy",
    "cmake",
    "fakeroot",
    "fish",
    "gcc",
    "gcc-multilib",
    "gdb",
    "git",
    "htop",
    "icecc",
    "icecream-sundae",
    "libboost-all-dev",
    "libcolamd2",
    "liblpsolve55-dev",
    "libncurses5",
    "libtinfo5",
    "lld",
    "lldb",
    "lp-solve",
    "make",
    "ninja-build",
    "parallel",
    "python3-pip",
    "time",
    "wget",
    "zsh"
]

In [ ]:
CMAKES = [
    'cmake',
    # ## 1. 编译器工具链选择 
    # (1) 指定gcc作为C语言的编译器，用于编译LLVM的C语言组件（如:运行时库、底层工具）;
    '-DCMAKE_C_COMPILER=gcc',
    # (2) 指定g++作为C++语言的编译器，用于编译LLVM的C++组件（如:LLVM IR库、前端接口、优化 passes）;
    '-DCMAKE_CXX_COMPILER=g++',

    # ## 2. 构建配置类型:
    # (1) 设置构建类型为Debug（调试模式）/Release（模式）。Debug模式下： 
    #   1）编译器会保留完整的调试符号（便于使用GDB等工具定位崩溃位置）;
    #   2）禁用大部分优化（避免优化导致的代码逻辑变化，便于调试）;
    '-DCMAKE_BUILD_TYPE=Debug', #Release

    # ## 3. 编译信息导出 
    # (1) 生成compile_commands.json文件（存放在构建目录中）,该文件记录每个源文件的完整编译命令（包括编译器标志、包含路径、宏定义）是Clangd（LLVM的C++语言服务器）等工具实现精准代码分析、自动补全的必要输入;
    '-DCMAKE_EXPORT_COMPILE_COMMANDS=1',

    # ## 4. 警告抑制
    # （1）禁用CMake的"开发者专用警告"（如未使用的CMake变量、过时的CMake脚本语法）。减少构建日志中的无关噪音，聚焦于关键错误;
    '-Wno-dev',
    # （2）禁用"建议使用override关键字"的警告（C++11特性，用于显式标记重写基类虚函数）。适用于维护旧代码（未全面使用override）的场景，避免过多警告干扰;
    '-Wno-suggest-override',

    # ## 5. LLVM核心功能配置
    # (1) 配置LLVM使用LLD（LLVM链接器）作为默认链接器（替代系统默认的ld.bfd或gold）。LLD是LLVM项目的一部分，针对LLVM IR优化，链接速度更快，尤其适合大型项目（如LLVM自身构建）。
    '-DLLVM_USE_LINKER=lld',
    # (2) 开启运行时类型信息（RTTI）支持。允许LLVM组件在运行时通过dynamic_cast获取对象的实际类型（例如，前端处理多态AST节点时），是部分LLVM passes（如代码生成）的必要功能。
    '-DLLVM_ENABLE_RTTI=ON',
    # (3) 开启异常处理（EH）支持。使LLVM生成的代码支持C++异常（如try-catch块），适用于需要异常处理的前端语言（如C++、Swift）。
    '-DLLVM_ENABLE_EH=ON',
    # (4) 开启断言检查（仅在Debug模式下有效）。在LLVM内部代码中插入assert语句，用于检测非法状态（如空指针解引用、数组越界），帮助开发者快速定位代码bug。 
    '-DLLVM_ENABLE_ASSERTIONS=ON',
    # (5) 排除LLVM基准测试套件的构建。基准测试用于评估LLVM各组件的性能（如IR优化、代码生成速度），对普通用户无用，关闭后可减少构建时间和磁盘占用。
    '-DLLVM_INCLUDE_BENCHMARKS=OFF',
    # (6) 指定LLVM支持的目标架构为ARM、RISC-V和X86。LLVM会根据此配置生成对应架构的代码生成器（Code Generator），仅编译需要的目标，缩短构建时间（例如，若无需支持PowerPC，可省略该架构）。
    '-DLLVM_TARGETS_TO_BUILD="ARM;RISCV;X86;AArch64"',
    # '-DLLVM_TARGETS_TO_BUILD=LLVM_ALL_TARGETS',

    # ## 6. 外部依赖配置
    # (1) 指向外部Clang（LLVM的C/C++前端）源码目录（版本由环境变量$CLANG_VER指定）。
    # LLVM会使用该目录下的Clang源码，而非从源码构建Clang，确保使用特定版本的Clang（避免版本冲突）。
    # '-DLLVM_EXTERNAL_CLANG_SOURCE_DIR=../dependencies/$CLANG_VER.src',
    # ../dependencies/$CLANG_EXTRA_VER.src
    f'-DLLVM_EXTERNAL_CLANG_SOURCE_DIR={CLANG_DIR}',
    # (2) 指定外部LLVMTA（LLVM TableGen辅助项目）的源码目录（位于当前构建目录的父目录）
    # LLVMTA用于生成LLVM TableGen文件（描述目标架构的指令集、寄存器等），集成自定义TableGen定义到LLVM构建中 
    # -DLLVM_EXTERNAL_LLVMTA_SOURCE_DIR=..
    f'-DLLVM_EXTERNAL_LLVMTA_SOURCE_DIR={TAM_HOME}',
    # (3) 声明llvmta为外部项目。
    # 告知CMake将该项目的源码纳入构建流程（编译其源码、链接其生成的库），确保LLVM能使用LLVMTA提供的功能。
    '-DLLVM_EXTERNAL_PROJECTS="llvmta"',
    # (4) ***
    # '-DLLVM_ENABLE_PROJECTS=clang;clang-tools-extra',

    # ## 7. 动态库构建
    # (1) 开启LLVM动态库（.so/.dll）的构建。
    # 将LLVM的核心组件(如IR库、优化 passes)编译为动态库，而非静态库。
    # '-DLLVM_BUILD_LLVM_DYLIB=ON',

    # ## 8. 构建系统选择 # 生成Ninja构建文件（而非传统的Makefile）。 
    # Ninja是专注于速度的构建系统，擅长处理大型项目的增量构建（仅编译修改过的文件），显著缩短LLVM的构建时间（尤其是后续的ninja命令执行阶段）。 
    '-GNinja',
    # -G "Unix Makefile"
    # -G "Unix Makefiles" 

    # ## 9. 源码目录与日志输出
    # 指定LLVM源码的根目录。该目录应包含LLVM的CMakeLists.txt文件（构建入口）。 
    LLVM_DIR
    # ../dependencies/$LLVM_VER.src
]

In [ ]:
with open(BASH_DIR, 'r+', encoding='utf-8') as _f:
    for _tc in [
        "export LD_LIBRARY_PATH=" + f"{LD_LIBRARY_PATH}",
        "export CPLUS_INCLUDE_PATH=" + f"{CPLUS_INCLUDE_PATH}", 
        "export PATH=${PATH}:" + os.path.join(BUILD_DIR, 'bin'),
        "export PATH=${PATH}:" + os.path.join(GUROBI_HOME, 'bin'),
        # "export GRB_LICENSE_FILE=/workspaces/llvmta/dependencies/gurobi.lic"

        ]:
        # for _l in _f:
        #     print(_l.split())
        if not any(_l.split() == _tc.split() for _l in _f):
            print(_tc.split())
            _f.write(f"{_tc}\n")


# # # # # # # # 
# echo "set completion-ignore-case on" >> /etc/inputrc
# # # # # # # # 
if os.system(f"bash -c 'source {BASH_DIR}'") != 0:
    exit(1)

# 1. 依赖库安装

In [ ]:
if os.system(f"echo {PASSWORD} | sudo -S apt-get update --fix-missing > /dev/null 2>&1") != 0:
    exit(1)

if os.system(f"echo {PASSWORD} | sudo -S apt-get install --no-install-recommends --fix-missing -y {' '.join(PAGES)} > /dev/null 2>&1") != 0:
    exit(1)

if os.system("pip3 install -i https://pypi.tuna.tsinghua.edu.cn/simple numpy > /dev/null 2>&1") != 0:
    exit(1)


# 2. 下载TA source文件

In [ ]:
# (a) TAM安装
if not os.path.exists(TAM_HOME):
    if os.system(f"git clone https://github.com/RTS-SYSU/Timing-Analysis-Multicores.git {TAM_HOME}") != 0:
        exit(1)

# (b) LLVMTA安装
# if not os.path.exists(LLVMTA_HOME):
#     if os.system(f"git clone https://gitlab.cs.uni-saarland.de/reineke/llvmta.git {LLVMTA_HOME}") != 0:
#         exit(1)

# 3. 下载lib文件

## (1) lp_solve

In [8]:
if not Path(LS_TDIR).exists():
    if os.system(f"sudo ln -s {LS_SDIR} {LS_TDIR}") != 0:
        exit(1)

## (2) gurobi

In [15]:
os.chdir(OPT_LIB)

# wget https://packages.gurobi.com/9.5/gurobi9.5.2_linux64.tar.gz
if not Path(GUROBI_FILE).exists():
    if os.system(f"sudo wget https://packages.gurobi.com/{GUROBI_VERS[0]}.{GUROBI_VERS[1]}/{GUROBI_PACKAGE}; \
                   sudo tar xfz {GUROBI_PACKAGE}; \
                   sudo rm {GUROBI_PACKAGE};") != 0:
        exit(1)
    
# # sudo ln -s /opt/gurobi952/linux64/lib/libgurobi95.so  /usr/lib/libgurobi.so 
if not os.path.exists(GB_TDIR):
    if os.system(f"sudo ln -s {GB_SDIR} {GB_TDIR}") != 0:
        exit(1)

In [ ]:
# a. arm
# wget https://developer.arm.com/-/media/Files/downloads/gnu/14.2.rel1/binrel/arm-gnu-toolchain-14.2.rel1-x86_64-arm-none-eabi.tar.xz
# sudo tar -xJf arm-gnu-toolchain-14.2.rel1-x86_64-arm-none-eabi.tar.xz -C /opt
# echo 'export PATH="/opt/arm-gnu-toolchain-14.2.rel1-x86_64-arm-none-eabi/bin:$PATH"' >> ~/.bashrc

# b. aarch64
# wget https://developer.arm.com/-/media/Files/downloads/gnu/14.2.rel1/binrel/arm-gnu-toolchain-14.2.rel1-x86_64-aarch64-none-elf.tar.xz
# sudo tar -xJf arm-gnu-toolchain-14.2.rel1-x86_64-aarch64-none-elf.tar.xz -C /opt
# echo 'export PATH="/opt/arm-gnu-toolchain-14.2.rel1-x86_64-aarch64-none-elf/bin:$PATH"' >> ~/.bashrc

# c. riscv
# wget https://static.dev.sifive.com/dev-tools/freedom-tools/v2020.12/riscv64-unknown-elf-toolchain-10.2.0-2020.12.8-x86_64-linux-ubuntu14.tar.gz
# sudo tar -xzf riscv64-unknown-elf-toolchain-*.tar.gz -C /opt
# echo 'export PATH="/opt/riscv64-unknown-elf-toolchain-10.2.0-2020.12.8-x86_64-linux-ubuntu14/bin:$PATH"' >> ~/.bashrc

# 4. 下载LLVM文件

## (1) all project

In [19]:
if not os.path.exists(LLVM_HOME):
    if os.system(f"git clone https://github.com/llvm/llvm-project.git -b llvmorg-{'.'.join(LLVM_VERS)} {LLVM_HOME}") != 0:
        exit(1)
else:
    os.chdir(LLVM_DIR)
    _patch_path = os.path.join(TAM_HOME, 'dependencies', 'patches', f'llvm-{".".join(LLVM_VERS)}.llvmta.diff')
    if os.system(f"patch -p1 < {_patch_path}") != 0:
        exit(1)
# Create patch with
# diff -ur $LLVM_VER.src  $LLVM_VER.src.patched > $LLVM_VER.llvmta.diff
# cd $LLVM_VER.src
# patch -p1 < ../../dependencies/patches/$LLVM_VER.llvmta.diff
# cd ../..

patching file CMakeLists.txt
patching file include/llvm/Analysis/LoopInfo.h
patching file include/llvm/Analysis/ScalarEvolution.h
patching file include/llvm/Analysis/ScalarEvolutionExpressions.h
patching file lib/Analysis/ScalarEvolution.cpp
patching file lib/CodeGen/MachineModuleInfo.cpp
patch unexpectedly ends in middle of line
Hunk #1 succeeded at 313 with fuzz 1.


## (2) llvm & clang

In [ ]:
if not os.path.exists(LLVM_HOME):
    os.mkdir(LLVM_HOME)
os.chdir(f"{LLVM_HOME}")

# if os.system(f"git clone https://github.com/llvm/llvm-project.git -b llvmorg-{'.'.join(LLVM_VERS)} LLVM") != 0:
#     exit(1)

# a. llvm
_llvm_file="llvm-14.0.6"
_llvm_path = os.path.join(LLVM_HOME, f"{_llvm_file}.src")
if not os.path.exists(_llvm_path):
    if os.system(f"wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{'.'.join(LLVM_VERS)}/{_llvm_file}.src.tar.xz; \
                    tar -xf {_llvm_file}.src.tar.xz; \
                    rm {_llvm_file}.src.tar.xz;") != 0:
        exit(1)
    else:
        os.chdir(_llvm_path)
        # patch -p1 < ../../dependencies/patches/$LLVM_VER.llvmta.diff 
        _patch_path = os.path.join(TAM_HOME, 'dependencies', 'patches', f'llvm-{".".join(LLVM_VERS)}.llvmta.diff')
        if os.system(f"patch -p1 < {_patch_path}") != 0:
            exit(1)
        os.chdir(f"{LLVM_HOME}")

# b. clang
_clang_file="clang-14.0.6"
_clang_path=os.path.join(LLVM_HOME, f"{_clang_file}.src")
if not os.path.exists(_clang_path):
    if os.system(f"wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{'.'.join(LLVM_VERS)}/{_clang_file}.src.tar.xz; \
                    tar -xf {_clang_file}.src.tar.xz; \
                    rm {_clang_file}.src.tar.xz") != 0:
        exit(1)

# 5. build

In [ ]:
print(" ".join(CMAKES))

cmake -DCMAKE_C_COMPILER=gcc -DCMAKE_CXX_COMPILER=g++ -DCMAKE_BUILD_TYPE=Debug -DCMAKE_EXPORT_COMPILE_COMMANDS=1 -Wno-dev -Wno-suggest-override -DLLVM_USE_LINKER=lld -DLLVM_ENABLE_RTTI=ON -DLLVM_ENABLE_EH=ON -DLLVM_ENABLE_ASSERTIONS=ON -DLLVM_INCLUDE_BENCHMARKS=OFF -DLLVM_TARGETS_TO_BUILD="ARM;RISCV;X86;AArch64" -DLLVM_EXTERNAL_CLANG_SOURCE_DIR=/home/fyj/Desktop/data/1_WCETSpace/LLVM/clang -DLLVM_EXTERNAL_LLVMTA_SOURCE_DIR=/home/fyj/Desktop/data/1_WCETSpace/TAM -DLLVM_EXTERNAL_PROJECTS="llvmta" -GNinja /home/fyj/Desktop/data/1_WCETSpace/LLVM/llvm


In [ ]:
if os.path.exists(BUILD_DIR):
    shutil.rmtree(BUILD_DIR)
os.mkdir(BUILD_DIR)
os.chdir(BUILD_DIR)

if os.system(" ".join(CMAKES)) != 0:
    exit(1)
# cmake $INSTRUCTION
# mv compile_commands.json ../compile_commands.json
# | tee -a $log_file：将CMake的输出同时显示在终端（实时查看配置进度）和追加到$log_file文件中（保存配置日志）。

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The ASM compiler identification is GNU
-- Found assembler: /bin/gcc
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /bin/gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for dlfcn.h
-- Looking for dlfcn.h - found
-- Looking for errno.h
-- Looking for errno.h - found
-- Looking for fcntl.h
-- Looking for fcntl.h - found
-- Looking for link.h
-- Looking for link.h - found
-- Looking for malloc/malloc.h
-- Looking for malloc/malloc.h - not found
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for signal.h
-- Looking for signal.h - found
-- Looking for sys/ioctl.h
--

1. 核心描述文件：CoreInformation.json:
    由于本项目针对的是多核处理器，因而需要提供核心描述文件，文件格式如下：

2. 循环描述文件：本项目对循环的解算无法做到完全的精确，因而为了确保测量的准确，请提供循环描述文件，文件格式如下：(具体来说，格式为)   

    <函数名>|<循环描述>|<循环的基本块描述>|<循环的迭代次数>

3. 生成的循环描述文件有两个，分别是:
- 'LoopAnnotations.csv'表示循环上界;
- 'LLoopAnnotations.csv'表示循环下界;

请根据实际情况，将两个描述文件分别填写完成:检查循环的边界是否设置正确，同时将那些标注为`-1`的循环修改为正确的循环边界。
修改完成后，将这两个文件放置到和测试用例同一目录下。

In [ ]:
os.chdir(BUILD_DIR)
if os.system(f"ninja") != 0:
    exit(1)

# /workspaces/llvmta/testcases/test/
# /workspaces/llvmta/testcases/src
# /workspaces/llvmta/testcases/test
# /workspaces/llvmta/testcases/test/minver
# /workspaces/llvmta/testcases/singletest/binarysearch

# /workspaces/Nuttx/nuttx/include
# /workspaces/Nuttx/apps/benchmarks/cachespeed

# cd testcases
# ./run.py -s <源代码所在目录> -o <输出文件目录> -p (判定是否打印循环描述文件)

# ./run.py -s /home/fyj/Desktop/data/TAM/testcases/test -o /home/fyj/Desktop/data/TAM/testcases/test -p


[1/3639] Building CXX object lib/Support/CMakeFiles/LLVMSupport.dir/ABIBreak.cpp.o
[2/3639] Building CXX object lib/Demangle/CMakeFiles/LLVMDemangle.dir/Demangle.cpp.o
[3/3639] Building CXX object lib/Demangle/CMakeFiles/LLVMDemangle.dir/DLangDemangle.cpp.o
[4/3639] Building CXX object lib/TableGen/CMakeFiles/LLVMTableGen.dir/TableGenBackendSkeleton.cpp.o
[5/3639] Building CXX object lib/Demangle/CMakeFiles/LLVMDemangle.dir/RustDemangle.cpp.o
[6/3639] Building CXX object lib/Demangle/CMakeFiles/LLVMDemangle.dir/MicrosoftDemangleNodes.cpp.o
[7/3639] Building CXX object lib/Support/CMakeFiles/LLVMSupport.dir/AArch64TargetParser.cpp.o
[8/3639] Building CXX object lib/Demangle/CMakeFiles/LLVMDemangle.dir/MicrosoftDemangle.cpp.o
[9/3639] Building CXX object lib/Support/CMakeFiles/LLVMSupport.dir/AutoConvert.cpp.o
[10/3639] Building CXX object lib/Support/CMakeFiles/LLVMSupport.dir/ARMBuildAttrs.cpp.o
[11/3639] Building CXX object lib/Support/CMakeFiles/LLVMSupport.dir/ARMWinEH.cpp.o
[12/363

# * Docker安装

In [ ]:
# (1) Docker安装
# docker build -t llvmtadocker:latest - < .devcontainer/Dockerfile

# (2) 运行
# a. linux 环境
# docker run -i -d -v {$TPATH}:/workspaces/llvmta:rw -v {$TPATH}:/workspaces/llvmta/build:rw --name XXX llvmtadocker:latest

# b. windows 环境
# docker run -i -d -v %cd%:/workspaces/llvmta:rw -v %cd%/build:/workspaces/llvmta/build:rw --name XXX llvmtadocker:latest